In [1]:
import cv2
import numpy as np
img = cv2.imread("hough.jpg", 0)
print(type(img))
img = np.asarray(img)

#Fetching the rows and columns
rows = len(img)
cols = len(img[0])

<class 'numpy.ndarray'>


# Sobel Operator

In [2]:
#initializing Sobel Operator
gx_sobel = [[-1,-2,-1],
           [0,0,0],
           [1,2,1]]

gy_sobel = [[-1,0,1],
           [-2,0,2],
           [-1,0,1]]

x_sobel = list(map(list,zip(*gx_sobel)))
y_sobel = list(map(list,zip(*gy_sobel)))


# Applying Gradient

In [3]:
#Used to pad the image
img_x = [[0]*902 for i in range(602)]
img_y= [[0]*902 for i in range(602)]

#Finding the Gradient to distinguish the images clearly. Especially the edges of the images
#Along X Axis

for i in range(1,img.shape[0]-1):
    for j in range(1,img.shape[1]-1):
        img_x[i][j] = img[i+1][j] - img[i-1][j]  #Subtract the neighbouring pixels on the x-axis

#Along Y Axis        
for i in range(1,img.shape[0]-1):
    for j in range(1,img.shape[1]-1):
        img_y[i][j] = img[i][j+1] - img[i][j-1]  #Subtract the neighbouring pixels on the y-axis

img_x = np.asarray(img_x)
img_y = np.asarray(img_y)

cv2.imwrite('T3_Images/Gradient_X.png',(img_x))
cv2.imwrite('T3_Images/Gradient_Y.png',(img_y))


C:\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in ubyte_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in ubyte_scalars
  from ipykernel import kernelapp as app


True

# Convolution

In [4]:
#Convolution
import math
sobelgx = [[0]*img.shape[1] for i in range(img.shape[0])]    #Creating empty 2D Lists
sobelgy = [[0]*img.shape[1] for i in range(img.shape[0])]
sobel = [[0]*img.shape[1] for i in range(img.shape[0])]
sobel_add = [[0]*img.shape[1] for i in range(img.shape[0])]

#Multiplying the sobel operator with the corresponding image pixel for that patch of the image to find the covoluted pixel value for (x,y)
for x in range(1, rows-1):
    for y in range (1, cols-1):
        gx =  x_sobel[0][0] * img[x-1][y-1] + x_sobel[0][1] * img[x][y-1] + x_sobel[0][2] * img[x+1][y-1] + (x_sobel[1][0] * img[x-1][y])   + (x_sobel[1][1] * img[x][y])   + (x_sobel[1][2] * img[x+1][y]) + (x_sobel[2][0] * img[x-1][y+1]) + (x_sobel[2][1] * img[x][y+1]) + (x_sobel[2][2] * img[x+1][y+1])
        sobelgy[x-1][y-1] = gx
        gy =  (y_sobel[0][0] * img[x-1][y-1]) + (y_sobel[0][1] * img[x][y-1]) + (y_sobel[0][2] * img[x+1][y-1]) + (y_sobel[1][0] * img[x-1][y])   + (y_sobel[1][1] * img[x][y])   + (y_sobel[1][2] * img[x+1][y]) + (y_sobel[2][0] * img[x-1][y+1]) + (y_sobel[2][1] * img[x][y+1]) + (y_sobel[2][2] * img[x+1][y+1])
        sobelgx[x-1][y-1] = gy
        
        sobel[x-1][y-1] = math.sqrt((gx*gx + gy*gy))   #Combining the outputs of the Sobel-X and Sobel-Y by Root of Squared Sum.
        sobel_add[x-1][y-1] = (gx + gy)/2              #Combining the outputs of the Sobel-x and Sobel-Y by adding the values

cv2.imwrite('T3_Images/Sobel_X.png',np.asarray(sobelgx)) 
cv2.imwrite('T3_Images/Sobel_Y.png',np.asarray(sobelgy))
cv2.imwrite('T3_Images/Sobel.png',np.asarray(sobel))
cv2.imwrite('T3_Images/Sobel_add.png',np.asarray(sobel_add))


True

# Normalization

In [5]:
theta = []
for i in range(0,360):
    theta.append(i)
theta = np.asarray(theta)

In [6]:
img = cv2.imread("T3_Images/Sobel_Y.png", 0)
print (img)
R = 22
thres_img = np.zeros((img.shape[0], img.shape[1]))
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if img[i][j] > 100:
            thres_img[i][j] = 255
thres_img = np.asarray(thres_img)
cv2.imwrite("T3_Images/T_sobel.jpg", thres_img)
thres_img.shape, img.shape

[[ 1  2  0 ...  0  0  0]
 [ 1  2  0 ... 11  0  0]
 [ 2  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  8  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


((477, 666), (477, 666))

In [7]:
def circumference_point(x,y, theta, R):
    points = []
    for t in theta:
        a = int((y - R*np.cos(math.radians(t))))
        b = int((x - R*np.sin(math.radians(t))))
        points.append([a,b])
    return points

In [16]:
def accumulator_matrix(thres_img,theta, R):
    
    acc = np.zeros((2*thres_img.shape[0], 2*thres_img.shape[1]))
    for x in range(thres_img.shape[0]):
        for y in range(thres_img.shape[1]):
            if thres_img[x][y] == 255:
                circum_points = circumference_point(x,y,theta,R)
                #print (circum_points)
                for point in circum_points:
                    acc[point[0]][point[1]] += 1
    cv2.imwrite("Bonus_accumulator.jpg",acc)
    return acc

In [17]:
import math
acc = accumulator_matrix(thres_img, theta, R)

In [10]:
cv2.imwrite("T3_Images/Bonus_acc.jpg",acc)

True

In [11]:
# sobelgx = cv2.imread("red_img.jpg", 0)
# cv2.imwrite("gray_red_img.jpg",sobelgx)

In [12]:
def max_indices(arr, k):
    '''
    Returns the indices of the k first largest elements of arr
    (in descending order in values)
    '''
    assert k <= arr.size, 'k should be smaller or equal to the array size'
    arr_ = arr.astype(float)  # make a copy of arr
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return convert(np.asarray(max_idxs))

In [13]:
def convert(k):
    edge_points = []
    for i in range(k.shape[0]):
        n = k[i][0].shape[0]
        if n>1:
            for j in range(n):
                edge_points.append([k[i][0][j], k[i][1][j]])
        else:
            edge_points.append([k[i][0][0], k[i][1][0]])
    return edge_points

In [14]:

e = []
edge_points = max_indices(acc, 60)
#edge_points
for i in range(len(edge_points)):
    if (i in range(50,56)):
        print (i)
    else:
        print (i)
        e.append(edge_points[i])
print (len(e), len(edge_points))
edge_points = e
len(edge_points)
e


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
156 162


[[519, 85],
 [520, 85],
 [518, 85],
 [142, 67],
 [47, 425],
 [518, 86],
 [519, 86],
 [141, 67],
 [426, 401],
 [142, 68],
 [141, 68],
 [143, 67],
 [46, 254],
 [425, 401],
 [278, 203],
 [46, 425],
 [47, 253],
 [253, 343],
 [279, 203],
 [48, 425],
 [252, 343],
 [46, 253],
 [604, 136],
 [517, 85],
 [519, 84],
 [47, 424],
 [605, 136],
 [520, 84],
 [521, 85],
 [46, 424],
 [603, 136],
 [280, 202],
 [379, 68],
 [252, 342],
 [520, 86],
 [426, 400],
 [253, 342],
 [378, 65],
 [532, 365],
 [48, 424],
 [378, 68],
 [378, 69],
 [279, 202],
 [381, 215],
 [45, 424],
 [605, 135],
 [531, 365],
 [143, 68],
 [602, 135],
 [45, 253],
 [532, 364],
 [377, 68],
 [382, 214],
 [49, 424],
 [273, 48],
 [381, 214],
 [427, 400],
 [608, 266],
 [47, 252],
 [271, 47],
 [517, 86],
 [65, 157],
 [272, 48],
 [383, 213],
 [425, 402],
 [604, 135],
 [45, 254],
 [270, 47],
 [377, 69],
 [609, 265],
 [272, 49],
 [382, 216],
 [606, 135],
 [48, 423],
 [254, 342],
 [379, 69],
 [424, 401],
 [521, 84],
 [45, 423],
 [46, 423],
 [251, 3

In [15]:
o_img = cv2.imread("hough.jpg")
print (edge_points[0][0], edge_points[0][1])
for i in range(len(edge_points)):
    cv2.circle(o_img, (edge_points[i][0],edge_points[i][1]), R, (0,255,0))
cv2.imwrite("T3_Images/circle.jpg",o_img)

519 85


True